In [1]:
import torch
import numpy as np
import imageio
from Model.VideoUNet import VideoUNet

In [2]:
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [3]:
stat_dict=torch.load("Model/checkpoint.pth")
model=VideoUNet(input_shape=[3,32,32],ncell=140,hiddensize=32)
model.load_state_dict(stat_dict)
model=model.eval()

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [4]:
test_input=torch.load("data/test_input")
test_input=test_input.float()
batch_size_test=int(test_input.shape[0]/64)
model=model.to(torch.device(device))
img_list=[]
test_size=test_input.shape[0]
with torch.no_grad():
    for batch_idx in range(0, test_size, batch_size_test):
        batch_test = test_input[batch_idx:batch_idx + batch_size_test].to(torch.device(device))
        test_result = model(batch_test)
        img=torch.mean(test_result,dim=0)
        img=img.permute(1,2,3,0).to(torch.device("cpu")).detach().numpy()
        img_list.append(img)
output_filename="result/"
for i in range (16):
    for j in range (4):
        imgs=img_list[i*4+j]
        name=(output_filename+"Train_stimuli"+str(i)+"_cue"+str(j)+"_frames")
        with imageio.get_writer(name+'.gif', mode='I') as writer:
            for img in imgs:
                writer.append_data(np.array(img*255, dtype=np.uint8))
if len(img_list)==64:
    print("success")

FileNotFoundError: [Errno 2] No such file or directory: 'data/test_input'

In [5]:
import matplotlib.pyplot as plt

i=5
j=5
imgs=img_list[i*4+j]
for img in imgs:
    plt.figure()
    plt.imshow(np.array(img*255, dtype=np.uint8))

NameError: name 'img_list' is not defined